In [1]:
import pandas as pd
import numpy as np
import collections.abc
# Koalas needs these mappings
collections.Iterable = collections.abc.Iterable
collections.Callable = collections.abc.Callable
import databricks.koalas as ks
import utils

# It seems you need to set this option for performance reasons.
# See: https://github.com/databricks/koalas/issues/1769 (it seems the issue is not only related to apply())
ks.set_option('compute.default_index_type', 'distributed')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/25 09:16:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
pandas_df = pd.read_csv('../datasets/yellow_tripdata_2015-01.csv')
koalas_df = ks.read_csv('../datasets/yellow_tripdata_2015-01.csv')

## Koalas Fails Here
First, Koalas doesn't support `.value_counts()` for the Koalas DataFrame. I tried the same I did for Dask which was manually call it for every column. But, for multiple columns, like the `tpep_pickup_datetime` or the `pickup_longitude` column, the Java backend runs out-of-memory and the process gets killed. And then it seems no Koalas operation works. So, we do `.value_counts()` on one column that works in Koalas, for both Koalas and Pandas.

In [3]:
%%time_cell
# The Koalas DataFrame doesn't seem to have value_counts(), so it seems we need to it for individual Series. This is a bit unfair
# because this loop could be done in parallel, but on the other hand, if it is that simple, then why Koalas doesn't do it?
print(koalas_df["VendorID"].value_counts())

2    6647797
1    6101189
Name: VendorID, dtype: int64


In [4]:
koalas_time = _TIMED_CELL
print(f"Koalas time: {koalas_time:.1f}s")

Koalas time: 2.5s


In [5]:
%%time_cell
print(pandas_df["VendorID"].value_counts())

2    6647797
1    6101189
Name: VendorID, dtype: int64


In [6]:
pandas_time = _TIMED_CELL
print(f"Pandas time: {pandas_time:.1f}s")

Pandas time: 0.1s


In [7]:
slowdown = koalas_time / pandas_time
utils.print_md(f"### Koalas is {slowdown:.1f}x slower.")

### Koalas is 42.6x slower.